# 如何在Colab上安裝Python虛擬環境及OpenVINO 2022.1
作者： Jack OmniXRI, 2022/09/05

Intel OpenVINO 2022.1提供很多安裝方式，在Google Colab上僅適用APT和PyPI方式安裝。本範例主要介紹PyPI安裝方式及安裝問題解決方式。

參考資料:   
[1] Intel, Install Intel® Distribution of OpenVINO™ Toolkit from PyPI Repository  
https://docs.openvino.ai/2022.1/openvino_docs_install_guides_installing_openvino_pip.html

[2] Intel, PyPI - 
OpenVINO Development Tools - openvino-dev  
https://pypi.org/project/openvino-dev/




## 1. 建立Python虛擬環境

註1：若沒有建立虛擬環境直接使用安裝 !pip installl openvino 會產生很多套件包衝突。  
註2：在虛擬環境執行任何指令都要加上 !source openvino_env/bin/activate; 若沒有加，則會跑回Google Colab原始環境。

In [1]:
# 安裝Python建立虛擬環境套件
!pip install virtualenv

# 建立虛擬環境名為 openvino_env
!virtualenv openvino_env

# 啟動虛擬環境並列出已安裝Python套件包清單
!source openvino_env/bin/activate; pip list

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 8.8 MB 7.1 MB/s 
     |████████████████████████████████| 468 kB 60.7 MB/s 
created virtual environment CPython3.7.13.final.0-64 in 1011ms
  creator CPython3Posix(dest=/content/openvino_env, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, wheel=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==22.2.2, setuptools==65.3.0, wheel==0.37.1
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator
Package    Version
---------- -------
pip        22.2.2
setuptools 65.3.0
wheel      0.37.1


## 2. 透過PyPI安裝OpenVINO到虛擬環境中

註：pip install openvino 只會安裝OpenVINO最精簡runtime套件僅供推論使用，但不包含下列套件，所以要改安裝 openvino-dev 
* Model Optimizer, Benchmark Application, Accuracy Checker, Post-Training Optimization Tool, Model Downloader, Model Converter, Model Quantizer, Model Information Dumper及OpenVINO版OpenCV。  

另外為了使用不同框架產生的模型，要加上對應框架參數（中括號內名稱），如openvino-dev[frame_name1,frame_name2,frame_name3...]，而支援的框架(farme_name)如下所示。
* caffe, kaldi, mxnet, onnx, pytorch, tensorflow, tensorflow2  

In [2]:
# 安裝OpenVINO開發者工具，含指定框架
!source openvino_env/bin/activate; pip install openvino-dev[onnx,pytorch,tensorflow2]

# 列出Python已安裝套件包清單
!source openvino_env/bin/activate; pip list

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 39.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 64.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.6/15.6 MB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.0/31.0 MB 45.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.3/22.3 MB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.8/99.8 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 76.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 68.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.8/298.8 kB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

## 3. 編寫openvino runtime和opencv測試碼
使用Google Colab魔術指令寫入python原始碼到文字檔中。  
列出OpenCV版本和可利用的硬體裝置清單。  

In [3]:
# 啟動Google Colab魔術指令將下列文字內容寫入test.py中
%%writefile test.py

# 導入OpenVINO Runtim和OpenCV套件包
import openvino.runtime as ov
import cv2

# 印出OpenCV版本
print(cv2.__version__)

# 建立運行核心並取得可使用之硬體裝置清單
core = ov.Core()
devices = core.available_devices

# 列出所有硬體裝置清單
for device in devices:
    device_name = core.get_property(device_name=device, name="FULL_DEVICE_NAME")
    print(f"{device}: {device_name}")

Writing test.py


In [4]:
# 列出 test.py 程式碼確認 （此步驟可略過）
!cat test.py


# 導入OpenVINO Runtim和OpenCV套件包
import openvino.runtime as ov
import cv2

# 印出OpenCV版本
print(cv2.__version__)

# 建立運行核心並取得可使用之硬體裝置清單
core = ov.Core()
devices = core.available_devices

# 列出所有硬體裝置清單
for device in devices:
    device_name = core.get_property(device_name=device, name="FULL_DEVICE_NAME")
    print(f"{device}: {device_name}")


## 4.在虛擬環境中執行測試程式
註：這裡顯示的OpenCV版本是虛擬環境中所安裝的版本，而不是Google Colab內建的OpenCV版本。  

In [5]:
# 在虛擬環境執行 Python 程式碼 test.py
!source openvino_env/bin/activate; python test.py

4.5.5
CPU:            Intel(R) Xeon(R) CPU @ 2.20GHz
